# Dataset Exploration

_Note: For this notebook to work you will need to have run the process dataset notebook or script._
_The file `data/processed/main_dataframe.csv` should exist._

In [22]:
import pandas as pd

In [23]:
df_main = pd.read_csv(
    "../data/processed/main_dataframe.csv",
    parse_dates=["date"]
)

In [24]:
df_main

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,station_code
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,66.0,54.23,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,SE
1,2020-01-01 01:00:00,2.11,NaN,NaN,NaN,19.0,57.0,NaN,735.7,0.0,96.0,5.4,0.010,11.20,8.1,NaN,SE
2,2020-01-01 02:00:00,2.06,NaN,NaN,NaN,19.0,68.0,53.84,734.8,0.0,96.0,5.5,0.010,11.26,5.5,NaN,SE
3,2020-01-01 03:00:00,1.96,NaN,NaN,NaN,19.0,68.0,36.47,734.2,0.0,96.0,5.4,0.010,11.35,3.8,NaN,SE
4,2020-01-01 04:00:00,1.98,NaN,NaN,NaN,16.0,48.0,33.59,733.9,0.0,96.0,5.5,0.010,11.47,3.3,NaN,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779376,2025-06-30 19:00:00,NaN,NaN,NaN,NaN,18.9,6.0,NaN,729.0,0.2,89.0,NaN,0.062,23.00,8.6,114.0,NE3
779377,2025-06-30 20:00:00,NaN,NaN,NaN,NaN,16.2,2.0,NaN,729.0,0.2,89.0,NaN,0.004,22.90,9.7,110.0,NE3
779378,2025-06-30 21:00:00,NaN,NaN,NaN,NaN,17.3,7.0,NaN,730.0,0.0,90.0,NaN,0.000,22.90,11.0,116.0,NE3
779379,2025-06-30 22:00:00,NaN,NaN,NaN,NaN,19.7,11.0,NaN,730.0,0.0,90.0,NaN,0.000,22.80,11.9,114.0,NE3
